In [ ]:
import requests, json, csv
import time, datetime, os
import pandas as pd

dt = datetime.datetime.now()
yearList = range(2016, dt.year+1)
monthList = range(1, 13)
isStock = True


def get_webmsg(yy, mm, sid):
    date = '%s%02d01' %(yy, int(mm))
    url = 'http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date='+date+'&stockNo='+sid
    res = requests.get(url)
    return res


def write_csv(sid, directory, filename, rows):
    outputFile = open(directory+filename, 'w', newline='', encoding='big5')
    outputWriter = csv.writer(outputFile)
    for row in rows:
        outputWriter.writerow(row)
    print(sid, filename, "done")
    outputFile.close()

    
def makedirs(sid):
    directory = 'D:/StockData/HistoryData/'+sid+'/'
    if not os.path.isdir(directory):
        os.makedirs(directory)


for stockId in range(1101, 10000):
    for year in yearList:
        for month in monthList:
            if (dt.year == year and month > dt.month):
                break
            sid = str(stockId)
            yy = str(year)
            mm = str(month)
            errorNum = 0
            while True:
                try:
                    res = get_webmsg(yy, mm, sid)
                    rows = csv.reader(res.text)
                    for row in rows:
                        if row == []:
                            isStock = False
                        break
                    if not isStock:
                        break
                    
                    directory = 'D:/StockData/HistoryData/'+sid+'/'
                    filename = yy+'%02d'%int(mm)+'.csv'
                    makedirs(sid)
                    write_csv(sid, directory, filename, rows)
                    errorNum = 0
                    break                        
                except:
                    errorNum += 1
                    print(sid, yy, mm, "error", errorNum, "times")
                    time.sleep(3)
                    if errorNum > 20:
                        isStock = False
                        break
                    continue
            if not isStock:
                break
            time.sleep(3)
        if not isStock:
            isStock = True
            print(sid, " no data")
            time.sleep(3)
            break

            
            
dlist1 = os.listdir('D:/StockData/HistoryData/')
del dlist1[-1]


for sid in dlist1:
    dlist2 = os.listdir('D:/StockData/HistoryData/'+sid+'/')
    del dlist2[-1]
    tempData = []
    
    for d in dlist2:
        try:
            op = open('D:/StockData/HistoryData/'+sid+'/'+d, 'r', encoding='big5')
            rows = csv.reader(op)
            tempData1 = []
            for row in rows:
                if row == []:
                    continue
                elif row == ['', '']:
                    continue
                tempData1.append(row)
            op.close()
            if not tempData1[0] == ['日期']:
                del tempData1[0]
            del tempData1[((len(tempData1)//9)*9):len(tempData1)]
            tempData2 = [[] for n in range(int(len(tempData1)/9))]
            for n in range(len(tempData1)):
                nt = int(n/9)
                tempData2[nt].append(tempData1[n][0])
            if not len(tempData) == 0:
                del tempData2[0]
            for data in tempData2:
                tempData.append(data)
        except:
            print(sid, d, 'error')
    
    #df = pd.DataFrame(tempData[1:], columns=tempData[0])
    #del df['成交筆數']
    #display(df)
    
    op = open('D:/StockData/ForAnalysis/'+sid+'.csv', 'w', newline='', encoding='utf-8-sig')
    writer = csv.writer(op)
    for row in tempData:
        writer.writerow(row)
    op.close()
    print(sid+'done')
    